In [534]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import plotly.express as px

Данные

In [535]:
path = '../data/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [536]:
df.iloc[:, :13].head(2)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский


In [537]:
# для подсчета количества пропущенных значений в каждом столбце
df.iloc[:, :13].isna().sum()

Unnamed: 0               0
ID  объявления           0
Количество комнат     1041
Тип                      0
Метро                 1315
Адрес                    0
Площадь, м2              0
Дом                      0
Парковка             13417
Цена                     0
Телефоны                 0
Описание                 0
Ремонт                2755
dtype: int64

In [538]:
df.fillna(0).isna().sum()

Unnamed: 0                  0
ID  объявления              0
Количество комнат           0
Тип                         0
Метро                       0
Адрес                       0
Площадь, м2                 0
Дом                         0
Парковка                    0
Цена                        0
Телефоны                    0
Описание                    0
Ремонт                      0
Площадь комнат, м2          0
Балкон                      0
Окна                        0
Санузел                     0
Можно с детьми/животными    0
Дополнительно               0
Название ЖК                 0
Серия дома                  0
Высота потолков, м          0
Лифт                        0
Мусоропровод                0
Ссылка на объявление        0
dtype: int64

In [539]:
df['Парковка'].value_counts()

Парковка
наземная          6043
подземная         2772
открытая          1017
многоуровневая     118
на крыше             1
Name: count, dtype: int64

In [540]:
# удаляем столбец 'Unnamed: 0'
df = df.drop('Unnamed: 0',axis = 1)
# удаляем Тип
df = df.drop('Тип', axis=1)
# убираем адреса не в Мск
df = df[df['Адрес'].str.contains('Москва')]
# Оставляем только те, где цена указана в рублях
df = df[df['Цена'].str.contains('руб.')].reset_index()


In [541]:
# Количество комнат: приведение к int и заполнение NaN 
df['Количество комнат'] = df['Количество комнат'].str.extract('(\d+)')
def func(x):
    try:
        return int(x)
    except:
        return np.NaN
df['rooms'] = df['Количество комнат'].map(func)
df.fillna({'rooms': 2}, inplace=True)

In [542]:
# преобразование цены 

def get_price(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  return int(r.split()[0])

def get_deposit(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  try:
      if int(r.split()[2]) < 1000 or int(r.split()[2]) > int(r.split()[0]) * 6:
        return int(r.split()[0])
      else:
        return int(r.split()[2])
  except:
      return int(r.split()[0])
    
x1 = df['Цена'].map(get_price) 
x2 = df['Цена'].map(get_deposit) 

In [543]:
# срок аренды
def extract_rent_term(text):
    match = re.search(r'Срок аренды - (Длительный|На несколько месяцев)', text)
    if match:
        return match.group(1)
    else:
        return None


In [544]:
# добавляем столбцы Цена за мес, залог и срок аренды
df['Срок аренды'] = df['Цена'].apply(extract_rent_term)
df.insert(9, "Цена за месяц",x1)
df.insert(10,'Залог',x2)
df.head(3)

,index,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Цена за месяц,Залог,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,rooms,Срок аренды
0,0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4.0,Длительный
1,1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4.0,Длительный
2,2,271173086,4,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,"+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4.0,Длительный


In [545]:
df['Срок аренды'].value_counts()

Срок аренды
Длительный              19162
На несколько месяцев      552
Name: count, dtype: int64

In [546]:
# преобразование этажа и этажности дома
def first_element(x):
    y = re.findall(r'\d+', x)
    return y[0]

df['Этаж'] = df['Дом'].map(first_element).astype('int')
def second_element(x):
    y = re.findall(r'\d+', x)
    return y[1]

df['Этажность дома'] = df['Дом'].apply(second_element).astype('int')
from operator import methodcaller
def func(x):
  return float(x[0])

In [547]:
# Общая площадь
def get_area_lamb(s):
    if '/' in s:
        # Если в строке есть '/', используем для извлечения первой части перед '/'
        return float(s.split('/')[0])
    else:
        # Если нет '/', преобразуем значение в число
        return float(s)

def get_area(data):
    return df['Площадь, м2'].astype(str).apply(get_area_lamb)

# Создаем новый столбец 
df['Общая площадь'] = get_area(df)

In [548]:
# вытаскиваем Станцию
def func (y):
    y[:y.find('(')]
    return y[:y.find('(')].strip()

def func2 (y):
    y[:y.find('(')]
    return y[y.find('м. ')+3:].strip()

x = df['Метро'].astype(str).map(func).map(func2).copy()
df.insert(3, "Станция", x)

In [549]:
# конвертируем расстояние до метро на пешком

def metro_time_lamb(s):
    try:
        if 'пешком' in s:
            # Если в строке содержится 'пешком'
            return float(s.split('(')[1].split(' мин')[0].strip()) if 'None' not in s else 0
        else:
            # Если не указано 'пешком', предполагаем, что это время на транспорте (умножаем на 10 для преобразования)
            return float(s.split('(')[1].split(' мин')[0].strip()) * 10 if 'None' not in s else 0

    except:
        pass

def get_metro_time(data):
    return df['Метро'].apply(metro_time_lamb)

# Создаем новый столбец 'metro_time' и заполняем его значениями, полученными из функции get_metro_time
df['Время пешком до метро, мин'] = get_metro_time(df)

In [550]:
df['Ремонт'].isna().sum()

2459

In [551]:
df.head(2)

,index,ID объявления,Количество комнат,Станция,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Цена за месяц,Залог,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,rooms,Срок аренды,Этаж,Этажность дома,Общая площадь,"Время пешком до метро, мин"
0,0,271271157,4,Смоленская,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4.0,Длительный,5,16,200.0,9.0
1,1,271634126,4,Смоленская,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",500000,500000,+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4.0,Длительный,5,16,198.0,8.0


In [552]:
df.drop(['index', 'Цена','Метро','Площадь, м2','Дом', 'Телефоны'], axis= 1, inplace=True)
df.head(2)

,ID объявления,Количество комнат,Станция,Адрес,Парковка,Цена за месяц,Залог,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,rooms,Срок аренды,Этаж,Этажность дома,Общая площадь,"Время пешком до метро, мин"
0,271271157,4,Смоленская,"Москва, улица Новый Арбат, 27",подземная,500000,500000,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4.0,Длительный,5,16,200.0,9.0
1,271634126,4,Смоленская,"Москва, улица Новый Арбат, 27",подземная,500000,500000,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4.0,Длительный,5,16,198.0,8.0


#### Метро 


In [553]:
metro = pd.read_csv('../data/list_of_moscow_metro_stations.csv', sep=',').rename(columns={'Station':'Станция'})
metro.head()

,Станция,metro,coor
0,Третьяковская,"Москва, станция метро Третьяковская","(55.738862, 37.6284034)"
1,Медведково,"Москва, станция метро Медведково","(55.886766, 37.6615351)"
2,Первомайская,"Москва, станция метро Первомайская","(55.7931157, 37.7999286)"
3,Калужская,"Москва, станция метро Калужская","(55.6550976, 37.5421849)"
4,Каховская,"Москва, станция метро Каховская","(55.6536252, 37.6054067)"


In [554]:
def func(s):
    s = s.replace('ё','е')
    s = s.replace('  ',' ')
    if s == 'Марк':
        s = 'Алтуфьево'
    elif s == 'Перерва':
        s = 'Братиславская'
    elif s.lower() == 'библиотека им. ленина':
        s = 'библиотека имени ленина'
    elif s.lower() == 'бескудниково':
        s = 'селигерская'
    elif s.lower() == 'гражданская':
        s = 'дмитровская'
    return s.lower().strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19714 entries, 0 to 19713
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID  объявления              19714 non-null  int64  
 1   Количество комнат           19179 non-null  object 
 2   Станция                     19714 non-null  object 
 3   Адрес                       19714 non-null  object 
 4   Парковка                    8551 non-null   object 
 5   Цена за месяц               19714 non-null  int64  
 6   Залог                       19714 non-null  int64  
 7   Описание                    19714 non-null  object 
 8   Ремонт                      17255 non-null  object 
 9   Площадь комнат, м2          12497 non-null  object 
 10  Балкон                      13091 non-null  object 
 11  Окна                        14569 non-null  object 
 12  Санузел                     17675 non-null  object 
 13  Можно с детьми/животными    148

In [555]:
#координаты

metro = metro.drop_duplicates('Станция').loc[:,['Станция','coor']]
metro.loc[:,'Станция'] = metro.loc[:,'Станция'].map(func)
df.loc[:,'Станция'] = df.loc[:,'Станция'].map(func)

m = df.merge(metro, how = 'left', on='Станция').dropna(how='all')
df.insert(3, "metro_coord",m['coor'])

In [557]:
df

,ID объявления,Количество комнат,Станция,metro_coord,Адрес,Парковка,Цена за месяц,Залог,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,rooms,Срок аренды,Этаж,Этажность дома,Общая площадь,"Время пешком до метро, мин"
0,271271157,4,смоленская,"(55.7488802, 37.5830392)","Москва, улица Новый Арбат, 27",подземная,500000,500000,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4.0,Длительный,5,16,200.0,9.0
1,271634126,4,смоленская,"(55.7488802, 37.5830392)","Москва, улица Новый Арбат, 27",подземная,500000,500000,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4.0,Длительный,5,16,198.0,8.0
2,271173086,4,смоленская,"(55.7488802, 37.5830392)","Москва, улица Новый Арбат, 27",подземная,500000,500000,"ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4.0,Длительный,5,16,200.0,7.0
3,272197456,4,смоленская,"(55.7488802, 37.5830392)","Москва, переулок Плотников, 21С1",подземная,400000,400000,ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4.0,Длительный,5,6,170.0,3.0
4,273614615,2,арбатская,"(55.7522109, 37.6087395)","Москва, улица Новый Арбат, 15",NaN,225000,225000,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2.0,Длительный,12,26,58.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19709,215565511,NaN,говорово,"(55.6564325, 37.4205042)","Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",NaN,42000,42000,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/215565511,2.0,Длительный,10,14,35.0,8.0
19710,274654844,1,солнцево,"(55.6564325, 37.4205042)","Москва, Производственная улица, 8к1, ш. Боровс...",NaN,45000,45000,Сдам однокомнатную квартиру на длительный срок...,Евроремонт,NaN,Лоджия (1),Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/274654844,1.0,Длительный,5,18,38.7,7.0
19711,268679909,2,солнцево,"(55.643751, 37.3912605)","Москва, Боровский проезд, 11",NaN,50000,50000,Рассматриваются квартиранты только славяне и т...,Дизайнерский,NaN,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/268679909,2.0,Длительный,5,5,43.1,6.0
19712,274807525,2,солнцево,"(55.6564325, 37.4205042)","Москва, улица Богданова, 6к1, ш. Боровское (3 ...",наземная,55000,50000,Двухкомнатная уютная квартира с видом во внутр...,Евроремонт,NaN,Лоджия (1),Во двор,"Совмещенный (1), Раздельный (1)",NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,2.65,Пасс (3),Да,https://www.cian.ru/rent/flat/274807525,2.0,Длительный,8,23,52.5,11.0
